# **Bonus Task:**

*References*:
- https://github.com/SMorettini/CNNs-on-CHB-MIT
- https://github.com/patschris/SeizureDetection
- Thesis referenced: https://pergamos.lib.uoa.gr/uoa/dl/object/2932363

## Data Preprocessing

In [ ]:
!pip install pyedflib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
from scipy import signal
import matplotlib.pyplot as plt
from pathlib import Path
from pyedflib import EdfReader
from tqdm.notebook import tqdm
from os import chdir, getcwd, listdir, mkdir, path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd '/content/drive/MyDrive/dsp_dataset/DSP Bonus Data/dl'

/content/drive/.shortcut-targets-by-id/1UwYBWUVUjop3jfFcjbVHNQRtkFuMPbQy/dsp_dataset/DSP Bonus Data/dl


In [ ]:
#!wget -r -N -c -np "https://physionet.org/files/chbmit/1.0.0/chb19/"

## EDF Reading:

In [ ]:
'''
https://pyedflib.readthedocs.io/en/latest/
'''
def readEdfFile(pathToFile, channels):
    f = EdfReader(pathToFile)
    n = f.signals_in_file
    signal_labels = f.getSignalLabels()
    signal_labels.append('seizure')
    sigbufs = np.zeros((f.getNSamples()[0],n+1))
    for i in np.arange(n): sigbufs[:,i] = f.readSignal(i)
    sigbufs[:, n]= 0.0
    df =  pd.DataFrame(data = sigbufs, columns = signal_labels)
    df = df.loc[:, channels]
    df = df.loc[:, ~df.columns.duplicated()]
    f._close()
    return df.values

In [ ]:
'''
https://www.mathworks.com/matlabcentral/answers/225716-how-i-can-read-chb01_03-edf-seizures-file-from-chb-mit-database-in-matlab-as-i-am-using-this-file-f
Returns start time and length of the seizure
'''
def get_seizure_period(file_location):
    bytes_array = []
    for b in Path(file_location).read_bytes(): bytes_array.append(b)
    return int(str(bin(bytes_array[38]))[2:] + str(bin(bytes_array[41]))[2:],2), bytes_array[49]

In [ ]:
'''
List of numpy array, each position contains a patient's array of data
'''

def read_and_store_data (dataset_folder, sample_rate, channels) :
    
    initial_path = getcwd()
    chdir(dataset_folder)
    
    patients = [d for d in listdir() if path.isdir(d) and d.startswith('chb')]
    patients.sort()
    arr = np.array([], dtype=np.float64).reshape(0, len(channels))
    seizurecount = 0
    for p in patients:
        chdir(p)

        print('Reading data of patient', p)
        
        # for each patient specify the edf files and the seizure files
        edf = [f for f in listdir() if path.isfile(f) and f.endswith('edf')]
        edf.sort()
        seizures = [f for f in listdir() if path.isfile(f) and f.endswith('seizures')]
        seizures.sort()
        for e in tqdm(edf):
            sigbufs = readEdfFile(e, channels)
            if seizures and seizures[0].startswith(e):
                (start, length) = get_seizure_period(seizures[0])
                for i in range(start * sample_rate, (start+length)*sample_rate + 1):
                    sigbufs[i][len(channels)-1] = 1.0
                    seizurecount = seizurecount +1
                seizures.pop(0)
        arr = np.concatenate([arr, sigbufs])
        chdir('..')
    chdir(initial_path)
    print(seizurecount)
    df = pd.DataFrame(arr, columns = channels)
    df.reset_index(drop = True, inplace = True)
    return df

In [ ]:
channels = ['FP1-F7', 'F7-T7','T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'seizure']


In [ ]:
readEdf = False
if readEdf:
    DF=read_and_store_data('/content/drive/MyDrive/dsp_dataset/DSP Bonus Data/dl', 256,channels)
    DF.to_csv('chb.csv', index = False)
else:
    print('Reading data from', 'chb.csv')
    DF = pd.read_csv('chb.csv', delimiter = ',', header = 0)

Reading data from chb.csv


In [ ]:
DF.shape, DF.columns

((3618304, 19),
 Index(['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3',
        'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8',
        'T8-P8', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'seizure'],
       dtype='object'))

In [ ]:
DF.head(6)

,FP1-F7,F7-T7,T7-P7,P7-O1,FP1-F3,F3-C3,C3-P3,P3-O1,FP2-F4,F4-C4,C4-P4,P4-O2,FP2-F8,F8-T8,T8-P8,P8-O2,FZ-CZ,CZ-PZ,seizure
0,62.710623,-180.317460,97.094017,-97.094017,22.075702,-11.526252,-28.717949,-99.438339,170.940171,-80.293040,24.810745,-105.689866,-1.367521,-42.783883,35.360195,18.559219,791.013431,43.565324,0.0
1,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,-0.195360,0.195360,0.0
2,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.976801,-0.586081,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,2.148962,1.367521,0.0
3,0.195360,0.586081,0.976801,0.195360,0.195360,0.195360,0.976801,0.195360,6.056166,-5.665446,0.195360,0.195360,3.321123,-2.930403,-0.195360,0.586081,16.605617,4.493284,0.0
4,0.586081,-0.586081,0.586081,0.195360,-0.195360,0.195360,0.195360,0.586081,0.195360,0.586081,0.195360,0.586081,1.758242,-1.367521,0.976801,0.586081,5.665446,-4.493284,0.0
5,0.195360,-1.758242,-2.539683,0.586081,0.195360,0.195360,-3.321123,0.195360,-25.201465,26.373626,-1.367521,-0.195360,-14.652015,13.870574,2.148962,-2.539683,-71.306471,-18.559219,0.0


###Trials

In [ ]:
# seizure_free_record_df1 = readEdfFile(pathToFile='/content/drive/My Drive/dsp_dataset/chb12_32.edf')
# seizure_free_record_df2 = readEdfFile(pathToFile='/content/drive/MyDrive/dsp_dataset/DSP Bonus Data/chb12/chb12_35.edf')

# seizure_record_df1 = readEdfFile(pathToFile='/content/drive/MyDrive/dsp_dataset/DSP Bonus Data/chb12/chb12_29.edf')
# seizure_record_df2 = readEdfFile(pathToFile='/content/drive/MyDrive/dsp_dataset/DSP Bonus Data/chb12/chb12_08.edf')


In [ ]:
# seizure_free_record_dfs=[seizure_free_record_df1,seizure_free_record_df2]#,seizure_free_record_df3]
# seizure_free_record_df=pd.concat(seizure_free_record_dfs)

In [ ]:
# seizure_record_dfs=[seizure_record_df1 , seizure_record_df2] #, seizure_record_df3]
# seizure_record_df=pd.concat(seizure_free_record_dfs)
# seizure_record_df['seizure']=seizure_record_df['seizure'].replace(0,1)

In [ ]:
 # add index column
# seizure_free_record_df['index'] = range(1, len(seizure_free_record_df) + 1)
# seizure_record_df['index'] = range(1, len(seizure_record_df) + 1)

In [ ]:
# seizure_free_record_df.head(5)

In [ ]:
# desing notch filter
samp_freq = 256  # Sample frequency (Hz)
notch_freq = 60.0  # Frequency to be removed from signal (Hz)
quality_factor = 20.0  # Quality factor
 
# Design a notch filter using signal.iirnotch
b_notch, a_notch = signal.iirnotch(notch_freq, quality_factor, samp_freq)
 
# Compute frequency response of the designed filter
freq, h = signal.freqz(b_notch, a_notch, fs=samp_freq)
 


In [ ]:
def filter_signal(b_notch,a_notch, selected_chanels, patent_record):
    # Apply notch filter to the noisy signal using signal.filtfilt
    filterd_signals = pd.DataFrame()
    n = np.linspace(0, 1, 256) # Generate 256 sample sequence in 1 sec as the fs is 256
    for chanel in selected_chanels:
        outputSignal = signal.filtfilt(b_notch, a_notch, patent_record[chanel] )
        filterd_signals[chanel] = outputSignal
        return filterd_signals

In [ ]:
# seizure_free_record_df.columns

In [ ]:
# selected_chanels_free=['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3',
#        'P3-O1', 'FZ-CZ', 'CZ-PZ', 'FP2-F4', 'F4-C4', 'C4-P4',
#        'P4-O2', 'FP2-F8', 'F8-T8', 'P7-T7',
#        'T7-FT9', 'FT9-FT10', 'FT10-T8']
# filter_seizure_free_record_df = filter_signal(b_notch, a_notch, selected_chanels_free, seizure_free_record_df)

In [ ]:
# seizure_record_df.sample(5)

In [ ]:
# seizure_record_df.columns

In [ ]:
# selected_chanels=['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3',
#        'P3-O1', 'FZ-CZ', 'CZ-PZ', 'FP2-F4', 'F4-C4', 'C4-P4',
#        'P4-O2', 'FP2-F8', 'F8-T8', 'P7-T7',
#        'T7-FT9', 'FT9-FT10', 'FT10-T8']

# filterd_seizure_record_df = filter_signal(b_notch, a_notch, selected_chanels, seizure_record_df)

In [ ]:
# dfs = [ seizure_free_record_df , seizure_record_df ] 
# full_record_df=pd.concat(dfs)

In [ ]:
# full_record_z_df = full_record_df
# for column in full_record_df.columns:
#   if column != 'index'or column != 'seizure' or column != '-':
#       full_record_z_df[column] = (full_record_df[column] -full_record_df[column].mean()) / full_record_df[column].std()  
#   else:
#       full_record_z_df[column] = full_record_df[column]   


In [ ]:
# full_record_z_df.drop('-', inplace=True, axis=1)

In [ ]:
# full_record_z_df.head(8)

## Data Processing


In [ ]:
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
!pip install pyentrp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pywt
import math
import numpy as np
import pandas as pd
from pyentrp import entropy
from tqdm.notebook import tqdm
from scipy.signal import welch
from scipy.integrate import simps
from scipy.stats import skew, kurtosis, variation
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import ClusterCentroids, RandomUnderSampler, NearMiss
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

### Features Processing:

### 

In [ ]:
def leftRightHemisphericChannels (df):
    ndf = pd.DataFrame()
    ndf['AvgLeftHemisphere'] = (df['F3-C3'] + df['C3-P3'])/2
    ndf['AvgRightHemisphere'] = (df['F4-C4'] + df['C4-P4'])/2
    ndf['seizure'] = df['seizure']
    return ndf

In [ ]:
'''
https://stackoverflow.com/questions/30272538/python-code-for-counting-number-of-zero-crossings-in-an-array
https://stackoverflow.com/questions/5613244/root-mean-square-in-numpy-and-complications-of-matrix-and-arrays-of-numpy
'''
def computeTimeDomainFeatures (x):
    mean = np.mean(x)
    var = np.var(x)
    sk = skew(x)
    kurt = kurtosis(x)
    std = np.std(x)
    median = np.median(x)
    zcr = ((x[:-1] * x[1:]) < 0).sum() / len(x)
    if x.mean() != 0:
        cv = variation(x)
    else:
        cv = math.nan
    if x.size > 0:
        rms = np.sqrt(x.dot(x)/x.size)
    else:
        rms = math.nan
    p2p = x.max() - x.min()
    sampEn = entropy.sample_entropy(x, 1)[0]
    return mean, var, sk, kurt, std, median, zcr, cv, rms, p2p, sampEn

In [ ]:
def computeCorrelation (left, right):
    return abs(np.correlate(left, right, 'full')).max()

In [ ]:
def featureExtractionLeftRight (df, sample_rate, step):
    print('Feature Extraction')
    ft = pd.DataFrame()
    c = 0
    for i in tqdm(range (0, df.shape[0], step)):
        temp = df.iloc[i:i+step]
        left = np.array(temp['AvgLeftHemisphere'])
        right = np.array(temp['AvgRightHemisphere'])

        # Time Domain Features
        ft.loc[c,'Lmean'], ft.loc[c,'Lvar'], ft.loc[c,'Lskew'],ft.loc[c,'Lkurt'], ft.loc[c,'Lstd'], ft.loc[c,'Lmedian'], ft.loc[c,'Lzcr'], ft.loc[c,'Lcv'], ft.loc[c,'Lrms'], ft.loc[c,'Lp2p'],ft.loc[c,'LsampEn'] = computeTimeDomainFeatures(left)
        ft.loc[c,'Rmean'], ft.loc[c,'Rvar'], ft.loc[c,'Rskew'],ft.loc[c,'Rkurt'], ft.loc[c,'Rstd'], ft.loc[c,'Rmedian'], ft.loc[c,'Rzcr'], ft.loc[c,'Rcv'], ft.loc[c,'Rrms'], ft.loc[c,'Rp2p'],ft.loc[c,'RsampEn'] = computeTimeDomainFeatures(right)

        # Frequency Domain Features
        ft.loc[c,'LdeltaPower'], ft.loc[c,'LthetaPower'], ft.loc[c,'LalphaPower'], ft.loc[c,'LbetaPower'], ft.loc[c,'LgammaPower'] = psd(left, sample_rate, left.shape[0])
        ft.loc[c,'RdeltaPower'], ft.loc[c,'RthetaPower'], ft.loc[c,'RalphaPower'], ft.loc[c,'RbetaPower'], ft.loc[c,'RgammaPower'] = psd(right, sample_rate, right.shape[0])

        
        # Correlation Features
        ft.loc[c, 'corr'] = computeCorrelation(left, right)

        ft.loc[c, 'seizure'] = temp['seizure'].value_counts().idxmax()
        c = c + 1
    return ft

In [ ]:
def featureExtractionAverage (df, sample_rate, step):
    print('Feature Extraction')
    ft = pd.DataFrame()
    c = 0
    for i in tqdm(range (0, df.shape[0], step)):
        temp = df.iloc[i:i+step]
        s = np.array(temp['surrogate'])
        
        # Time Domain Features
        ft.loc[c,'mean'], ft.loc[c,'var'], ft.loc[c,'skew'],ft.loc[c,'kurt'], ft.loc[c,'std'], ft.loc[c,'median'], ft.loc[c,'zcr'], ft.loc[c,'cv'], ft.loc[c,'rms'], ft.loc[c,'p2p'],ft.loc[c,'sampEn'] = computeTimeDomainFeatures(s)
        
        # Frequency Domain Features
        ft.loc[c,'deltaPower'], ft.loc[c,'thetaPower'], ft.loc[c,'alphaPower'], ft.loc[c,'betaPower'], ft.loc[c,'gammaPower'] = psd(s, sample_rate, s.shape[0])
        
        ft.loc[c, 'seizure'] = temp['seizure'].value_counts().idxmax()
        c = c + 1
    return ft

In [ ]:
def featureExtractionFull (df, sample_rate, step):
    print('Feature Extraction')
    ft = pd.DataFrame()
    c = 0
    for i in tqdm(range (0, df.shape[0], step)):
        temp = df.iloc[i:i+step]
        for j in range(0, df.shape[1]-1):
            s = np.array(temp.iloc[:, j])

            # Time Domain Features
            ft.loc[c, 'mean'+str(j)], ft.loc[c, 'var'+str(j)], ft.loc[c, 'skew'+str(j)],ft.loc[c, 'kurt'+str(j)], ft.loc[c, 'std'+str(j)], ft.loc[c, 'median'+str(j)], ft.loc[c, 'zcr'+str(j)], ft.loc[c, 'cv'+str(j)], ft.loc[c, 'rms'+str(j)], ft.loc[c, 'p2p'+str(j)],ft.loc[c, 'sampEn'+str(j)] = computeTimeDomainFeatures(s)

            # Frequency Domain Features
            ft.loc[c, 'deltaPower'+str(j)], ft.loc[c, 'thetaPower'+str(j)], ft.loc[c, 'alphaPower'+str(j)], ft.loc[c, 'betaPower'+str(j)], ft.loc[c, 'gammaPower'+str(j)] = psd(s, sample_rate, s.shape[0])

        ft.loc[c, 'seizure'] = temp['seizure'].value_counts().idxmax()
        c = c + 1
    return ft

In [ ]:
'''
Compute the average bandpower of an EEG signal
https://raphaelvallat.com/bandpower.html
'''
def psd (x, fs, win):
    bands = [0.5, 4, 8, 12, 30, 100]
    freqs, psd = welch(x, fs, nperseg = win)
    avg_power=[]
    while len(bands)>1:
        idx = np.logical_and(freqs >= bands[0], freqs <= bands[1])
        power_simps = simps(psd[idx], dx=bands[1]-bands[0])
        avg_power.append(power_simps)
        bands = np.copy(bands[1:])
    for p in avg_power:
        yield p

In [ ]:
def averageChannels (df):
    ndf = pd.DataFrame()
    ndf['surrogate'] = df.iloc[:, :df.shape[1]-1].mean(axis=1)
    ndf['seizure'] = df['seizure']
    return ndf

In [ ]:
def dimentionalityReduction(features, threshold):
    ft = features.iloc[:, :features.columns.size-1]
    pca_ft = pd.DataFrame(PCA(n_components = threshold).fit_transform(ft))
    pca_ft['seizure'] = features['seizure'].copy()
    return pca_ft

In [ ]:
def featureNormalization(ft):
    scaled_df = StandardScaler().fit_transform(ft.iloc[:, :ft.shape[1]-1])
    norm_ft = pd.DataFrame(scaled_df)
    norm_ft['seizure'] = ft['seizure'].copy()
    return norm_ft

In [ ]:
def removeNonNumericValues(df):
    df.replace([np.inf, -np.inf], np.nan, inplace = True)
    df.dropna(inplace = True)

In [ ]:
def oversamplingSMOTE(ft, ft_index, neighbors):
    smote = SMOTE(sampling_strategy = 'minority', k_neighbors = neighbors)
    smote_features, smote_indicator = smote.fit_resamplele(ft, ft_index)
    smote_features['seizure'] = smote_indicator
    return smote_features

In [ ]:
def oversamplingADASYN(ft, ft_index, neighbors):
    adasyn = ADASYN(sampling_strategy='minority', n_neighbors = neighbors)
    adasyn_features, adasyn_indicator = adasyn.fit_resample(ft, ft_index)
    adasyn_features['seizure'] = adasyn_indicator
    return adasyn_features

In [ ]:
def minorityOversampling (ft, ft_index, neighbors, method):
    if method.upper() == 'ADASYN':
        return oversamplingADASYN(ft, ft_index, neighbors)
    else:
        return oversamplingSMOTE(ft, ft_index, neighbors)

In [ ]:
def undersamplingClusterCentroids(ft, ft_index, rate):
    cc = ClusterCentroids(sampling_strategy = rate)
    cc_features, cc_indicator = cc.fit_resample(ft, ft_index)
    cc_features['seizure'] = cc_indicator
    return cc_features

In [ ]:
def undersamplingNearMiss(ft, ft_index, rate, neighbors):
    nm = NearMiss(sampling_strategy=rate, n_neighbors=neighbors)
    nm_features, nm_indicator = nm.fit_resample(ft, ft_index)
    nm_features['seizure'] = nm_indicator
    return nm_features

In [ ]:
def undersamplingRandom (ft, ft_index, rate):
    ru = RandomUnderSampler(sampling_strategy = rate)
    ru_features, ru_indicator = ru.fit_resample(ft, ft_index)
    ru_features['seizure'] = ru_indicator
    return ru_features

In [ ]:
def majorityUndersampling (ft, ft_index, rate, neighbors, method):
    if method.upper() == 'RANDOM':
        return undersamplingRandom (ft, ft_index, rate)
    elif method.upper() == 'NEARMISS':
        return undersamplingNearMiss(ft, ft_index, rate, neighbors)
    else:
        return undersamplingClusterCentroids(ft, ft_index, rate)

#### Features Extraction:

In [ ]:
def featureExtraction (df, sample_rate, step, pca_tolerance, undersampling_method, undersampling_rate, undersampling_neighbors, oversampling_method, oversampling_neighbors, exp):
    if exp.upper() not in ['FULL', 'AVERAGE', 'LEFTRIGHT']:
        print('No such experiment:', exp)
        return
    else:
        print ('Executing Experiment', exp)
    if exp.upper() == 'FULL':
        ft = pd.DataFrame(featureExtractionFull (df, sample_rate, step))
    elif exp.upper() == 'AVERAGE':
        ft = pd.DataFrame(featureExtractionAverage (averageChannels(df), sample_rate, step))
    else:
        ft = pd.DataFrame(featureExtractionLeftRight (leftRightHemisphericChannels(df), sample_rate, step))
    removeNonNumericValues(ft)
    ft = featureNormalization(ft)
    print('Normalized features')
    removeNonNumericValues(ft)
    size = ft.shape
    print('Reducing features dimension')
    ft = dimentionalityReduction(ft, pca_tolerance)
    removeNonNumericValues(ft)
    print('Dimensions reduced from', size, 'to', ft.shape)
    size = ft.seizure.value_counts()
    print('Undersampling the majority class using', undersampling_method)
    ft = majorityUndersampling(ft.loc[:, ft.columns != 'seizure'], ft['seizure'], undersampling_rate, undersampling_neighbors, undersampling_method)
    removeNonNumericValues(ft)
    print('Majority class downsampled from (', size[0], ', ', ft.shape[1], ') to ', ft.shape, sep = '')
    size = ft.shape
    print('Oversampling the minority class using', oversampling_method)
    ft = minorityOversampling(ft.loc[:, ft.columns != 'seizure'], ft['seizure'], oversampling_neighbors, oversampling_method)
    ft = shuffle(ft)
    ft.reset_index(drop = True, inplace = True)
    removeNonNumericValues(ft)
    print('Minority class upsampled from (', size[0], ', ', ft.shape[1], ') to ', ft.shape, sep='')
    print('Writing features to a csv file\n')
    ft.to_csv(exp + 'Features.csv', index = False)
    return ft

In [ ]:
def createTrainingAndTestDatasets(dataset, test_ratio):
    X = dataset.loc[:, dataset.columns != 'seizure']
    y = dataset['seizure']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_ratio, shuffle = True)
    return X_train, X_test, y_train, y_test

In [ ]:
def trainTestData (features, test_ratio, k_fold, perfInd):
    X_train, X_test, y_train, y_test = createTrainingAndTestDatasets(features, test_ratio)
    results = pd.DataFrame(columns = perfInd)
    kf = KFold(n_splits = k_fold, shuffle = True)
    return X_train, X_test, y_train, y_test, results, kf

#### Different Classification Models:

In [ ]:
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from keras.regularizers import l2
from keras.models import Sequential
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, LSTM
import time
from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix, f1_score, matthews_corrcoef
from unittest import result

In [ ]:
def classificationPerformanceIndexes (true_arr, pred_arr, time):
    (tn, fp, fn, tp) = confusion_matrix(true_arr, pred_arr).ravel()
    acc = accuracy_score(true_arr, pred_arr)                           # Accuracy
    snv = tp/(tp + fn)                                                 # Sensitivity or True Positive Rate (TPR)
    spc = tn/(tn + fp)                                                 # Specificity or True Negative Rate (TNR)
    ppv = tp/(tp + fp)                                                 # Precision or Positive Predictive Value (PPV)
    f1 = f1_score(true_arr, pred_arr)                                  # F1 score
    mcc = matthews_corrcoef(true_arr, pred_arr)                        # Matthews Correlation Coefficient
    kappa = cohen_kappa_score(true_arr, pred_arr)                      # Cohen’s Kappa    
    return acc, snv, spc, ppv, f1, mcc, kappa, time
'''
TP : the number of segments that are correctly identified as ictal (x_true == x_pred == 1)
TN : the number of EEG segments that are correctly classified as non-ictal (x_true == x_pred == 0)
FP : the number of EEG segments that are incorrectly classified as ictal (x_true == 0 && x_pred == 1)
FN : the segments that are incorrectly classified as non-ictal (x_true == 1 && x_pred == 0)
'''

'\nTP : the number of segments that are correctly identified as ictal (x_true == x_pred == 1)\nTN : the number of EEG segments that are correctly classified as non-ictal (x_true == x_pred == 0)\nFP : the number of EEG segments that are incorrectly classified as ictal (x_true == 0 && x_pred == 1)\nFN : the segments that are incorrectly classified as non-ictal (x_true == 1 && x_pred == 0)\n'

In [ ]:
def printClassificationPerformanceIndexes(method, acc, snv, spc, ppv, f1, mcc, kappa):
    print('Method:', method)
    print('Accuracy:', acc)
    print('Sensitivity/Recall:', snv)
    print('Specificity:', spc)
    print('Precision:', ppv)
    print('F1 Score:', f1)
    print('Matthews Correlation Coefficient:', mcc)
    print('Cohen’s Kappa:', kappa)

In [ ]:
def TrainingKfold (X, train, test):
    X_train = X.iloc[train,:X.shape[1]-1]
    y_train = X.loc[train,'seizure']
    X_test = X.iloc[test,:X.shape[1]-1]
    y_test = X.loc[test,'seizure']
    return X_train, y_train, X_test, y_test

##### SVM:


In [ ]:
def SVM_Classifier():
    return SVC(kernel='poly', gamma='auto', degree=3)

In [ ]:
def SVM(X_train, y_train, X_test, y_test, results):
    print('Implementing SVM method...')
    start = time.time()
    clf = SVM_Classifier()
    svm_ind = clf.fit(X_train, y_train).predict(X_test)
    end = time.time()
    t = round(end - start,2)
    acc, snv, spc, ppv, f1, mcc, kappa, tt = classificationPerformanceIndexes (y_test, svm_ind, t)
    results.loc['SVM', :] = acc, snv, spc, ppv, f1, mcc, kappa, t
    printClassificationPerformanceIndexes('SVM', acc, snv, spc, ppv, f1, mcc, kappa)
    print('SVM finished in', t, 'sec\n')

In [ ]:
def SVM_Kfold(X, kf, cols, results):
    f = pd.DataFrame(columns = cols)
    print('Implementing SVM k-fold...')
    start = time.time()
    clf = SVM_Classifier()
    for train, test in kf.split(X):
        X_train, y_train, X_test, y_test = TrainingKfold (X, train, test)
        svm_ind = clf.fit(X_train, y_train).predict(X_test)
        f.loc[f.shape[0], :] = classificationPerformanceIndexes (y_test, svm_ind, 0)
    end = time.time()
    t = round(end - start,2)    
    acc, snv, spc, ppv, f1, mcc, kappa, tt = np.array(f.mean(axis=0))
    results.loc['SVM Kfold', :] = acc, snv, spc, ppv, f1, mcc, kappa, t
    printClassificationPerformanceIndexes('SVM Kfold', acc, snv, spc, ppv, f1, mcc, kappa)
    print('SVM k-fold finished in', t, 'sec\n')

In [ ]:
def CompleteSVM(train_dat, test_dat, train_ind, test_ind, results, features, kf, perfInd):
    SVM(train_dat, train_ind, test_dat, test_ind, results)
    SVM_Kfold(features, kf, perfInd, results)

##### K-NN:


In [ ]:
def KNN_Classifier():
    return KNeighborsClassifier(n_neighbors = 3, weights='distance', metric = 'manhattan', n_jobs = -1)

In [ ]:
def KNN(X_train, y_train, X_test, y_test, experiment,results):
    print('Implementing KNN...')
    start = time.time()
    clf = KNN_Classifier()
    knn_ind = clf.fit(X_train, y_train).predict(X_test)
    end = time.time()
    t = round(end - start,2)    
    acc, snv, spc, ppv, f1, mcc, kappa, tt = classificationPerformanceIndexes (y_test, knn_ind, t)
    results.loc['KNN', :] = acc, snv, spc, ppv, f1, mcc, kappa, t
    printClassificationPerformanceIndexes('KNN', acc, snv, spc, ppv, f1, mcc, kappa)
    print('KNN finished in', t,'sec\n')

In [ ]:
def KNN_Kfold(X, experiment, kf, cols, results):
    f = pd.DataFrame(columns = cols)
    print('Implementing KNN k-fold...')
    start = time.time()
    clf = KNN_Classifier()
    for train, test in kf.split(X):
        X_train, y_train, X_test, y_test = TrainingKfold (X, train, test)
        knn_ind = clf.fit(X_train, y_train).predict(X_test)
        f.loc[f.shape[0], :] = classificationPerformanceIndexes (y_test, knn_ind, 0)
    end = time.time()
    t = round(end - start,2)    
    acc, snv, spc, ppv, f1, mcc, kappa, tt =  np.array(f.mean(axis=0))
    printClassificationPerformanceIndexes('KNN Kfold', acc, snv, spc, ppv, f1, mcc, kappa)
    results.loc['KNN Kfold', :] = acc, snv, spc, ppv, f1, mcc, kappa, t
    print('KNN k-fold finished in', t,'sec\n')

In [ ]:
def CompleteKNN(train_dat, test_dat, train_ind, test_ind, results, experiment, features, kf, perfInd):
    KNN(train_dat, train_ind, test_dat, test_ind, experiment, results)
    KNN_Kfold(features, experiment, kf, perfInd, results)

##### Logistic Regression:


In [ ]:
def LogRef_Classifier():
    return LogisticRegression(penalty = 'l2', C = 0.1, max_iter = 200, solver = 'lbfgs') 

In [ ]:
def LogReg(X_train, y_train, X_test, y_test, results, experiment):
    print('Implementing Logistic Regression...')
    start = time.time()
    clf = LogRef_Classifier()    
    lr_ind = clf.fit(X_train, y_train).predict(X_test)
    end = time.time()
    t = round(end - start,2)    
    acc, snv, spc, ppv, f1, mcc, kappa, tt = classificationPerformanceIndexes (y_test, lr_ind, t)
    results.loc['Logistic Regression', :] = acc, snv, spc, ppv, f1, mcc, kappa, t
    printClassificationPerformanceIndexes('Logistic Regression', acc, snv, spc, ppv, f1, mcc, kappa)
    print('Logistic Regression finished in', t,'sec\n')

In [ ]:
def LogReg_Kfold(X, kf, cols, results, experiment):
    f = pd.DataFrame(columns = cols)
    print('Implementing Logistic Regression k-fold...')
    start = time.time()
    clf = LogRef_Classifier()
    for train, test in kf.split(X):
        X_train, y_train, X_test, y_test = TrainingKfold (X, train, test)
        lr_ind = clf.fit(X_train, y_train).predict(X_test)
        f.loc[f.shape[0], :] = classificationPerformanceIndexes (y_test, lr_ind, 0)
    end = time.time()
    t = round(end - start,2)
    acc, snv, spc, ppv, f1, mcc, kappa, tt = np.array(f.mean(axis=0))
    results.loc['Logistic Regression Kfold', :] = acc, snv, spc, ppv, f1, mcc, kappa, t
    printClassificationPerformanceIndexes('Logistic Regression Kfold', acc, snv, spc, ppv, f1, mcc, kappa)
    print('Logistic Regression k-fold finished in', t,'sec\n')

In [ ]:
def CompleteLR(train_dat, test_dat, train_ind, test_ind, results, experiment, features, kf, perfInd):
    LogReg(train_dat, train_ind, test_dat, test_ind, results, experiment)
    LogReg_Kfold(features, kf, perfInd, results, experiment)

##### LSTM

In [ ]:
def LstmModel (size, lstm_units, dense_units, dropout_percentage, loss_function, metric):
    model = Sequential()
    model.add(LSTM(lstm_units, recurrent_regularizer = l2(1e-2), activity_regularizer = l2(1e-4), bias_regularizer = l2(1e-6)))
    model.add(Dropout(dropout_percentage))
    model.add(Dense(dense_units, activation = 'relu', kernel_regularizer = l2(1e-3), bias_regularizer = l2(1e-2)))
    model.add(Dropout(dropout_percentage/2))
    model.add(Dense(1, activation = 'sigmoid', kernel_regularizer = l2(1e-3), bias_regularizer = l2(1e-2)))
    model.compile(optimizer = Adam(learning_rate = 1e-3), loss = loss_function, metrics = metric)
    return model

In [ ]:
def LSTM_method (model, X_train, y_train, X_test, y_test, batch, epochs, results):
    print('Implementing LSTM...')
    start = time.time()
    es = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 5, mode = 'auto', restore_best_weights = True, verbose = 0)
    history = model.fit(X_train, y_train, batch_size = batch, epochs = epochs, validation_data = (X_test,y_test), callbacks = es, verbose = 0)
    lstm_ind = (model.predict(X_test, batch_size = batch) >= 0.5).astype('int')
    end = time.time()
    t = round(end - start,2)
    acc, snv, spc, ppv, f1, mcc, kappa, tt = classificationPerformanceIndexes (y_test, np.reshape(lstm_ind, lstm_ind.shape[0]), t)
    results.loc['LSTM', :] = acc, snv, spc, ppv, f1, mcc, kappa, t
    printClassificationPerformanceIndexes('LSTM', acc, snv, spc, ppv, f1, mcc, kappa)
    print('LSTM finished in', t,'sec\n')

In [ ]:
def LSTM_method_Kfold(X, kf, cols, model, batch, epochs, results):
    f = pd.DataFrame(columns = cols)
    print('Implementing LSTM k-fold...')
    start = time.time()
    es = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 5, mode = 'auto', restore_best_weights = True, verbose = 0)
    for train, test in kf.split(X):
        X_train = X.iloc[train,:X.shape[1]-1]
        X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
        y_train = X.loc[train,'seizure'].values.astype(int)
        X_test = X.iloc[test,:X.shape[1]-1]
        X_test = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))
        y_test = X.loc[test,'seizure'].values.astype(int)
        history = model.fit(X_train, y_train, batch_size = batch, epochs = epochs, validation_data = (X_test,y_test), callbacks = es, verbose = 0)
        lstm_ind = (model.predict(X_test, batch_size = batch) >= 0.5).astype('int')
        f.loc[f.shape[0], :] = classificationPerformanceIndexes (y_test, np.reshape(lstm_ind, lstm_ind.shape[0]), 0)
    end = time.time()
    t = round(end - start,2)
    acc, snv, spc, ppv, f1, mcc, kappa, tt = np.array(f.mean(axis=0))
    results.loc['LSTM Kfold', :] = acc, snv, spc, ppv, f1, mcc, kappa, t
    printClassificationPerformanceIndexes('LSTM Kfold', acc, snv, spc, ppv, f1, mcc, kappa)
    print('LSTM finished in', t,'sec\n')

In [ ]:
def CompleteLSTM (train_dat, test_dat, train_ind, test_ind, results, ft, kf, perfInd, epochs, batch, lstm_units, dense_units, dropout_percentage, loss_function, metric):
    X_train = np.reshape(train_dat.values, (train_dat.shape[0], 1, train_dat.shape[1]))
    y_train = train_ind.values.astype(int)
    X_test = np.reshape(test_dat.values, (test_dat.shape[0], 1, test_dat.shape[1]))
    y_test = test_ind.values.astype(int)

    lstm_model = LstmModel (train_dat.shape[1], lstm_units, dense_units, dropout_percentage, loss_function, metric)
    LSTM_method (lstm_model, X_train, y_train, X_test, y_test, batch, epochs, results)
    LSTM_method_Kfold (ft, kf, perfInd, lstm_model, batch, epochs, results)

## Results:

#### Experimenting:

In [ ]:
experiments = ['Average', 'LeftRight', 'Full']
perfInd = ['Accuracy', 'Sensitivity', 'Specificity', 'Precision', 'F1 Score', 'MCC', 'Kappa', 'Time']

In [ ]:

sample_rate = 256
time_window = 2
step = time_window * sample_rate
k_fold=5
test_ratio=0.31415
epochs =100
batch= 10
dropout_percentage = 0.1
loss_function = 'mean_squared_error'
metric = 'accuracy'
experiments = ['Average', 'LeftRight', 'Full']
undersampling_method = 'ClusterCentroids'
undersampling_rate = 0.2
undersampling_neighbors = 3
oversampling_method = 'ADASYN'
oversampling_neighbors = 11
for exp in experiments:
  if exp.upper() not in ['FULL', 'AVERAGE', 'LEFTRIGHT']:
        print('No such experiment:', exp)
  else:
        print ('Executing Experiment', exp)
  FtDFres= featureExtraction (DF, 256, step, 0.9 , undersampling_method, undersampling_rate, undersampling_neighbors, oversampling_method, oversampling_neighbors, exp)
  FtDFres.to_csv(exp + '.csv')
  X_train, X_test, y_train, y_test, results, kf = trainTestData (FtDFres, test_ratio, k_fold, perfInd)  
  CompleteSVM(X_train, X_test, y_train, y_test, results, FtDFres, kf, perfInd)
  CompleteKNN(X_train, X_test, y_train, y_test, results, exp, FtDFres, kf, perfInd)
  CompleteLR(X_train, X_test, y_train, y_test, results, exp, FtDFres, kf, perfInd)
  if exp.upper() == 'AVERAGE':
        lstm_units = 32
        dense_units = 8
  elif exp.upper() == 'LEFTRIGHT':
        lstm_units = 64
        dense_units = 16
  else:
        lstm_units = 128
        dense_units = 32 
  CompleteLSTM(X_train, X_test, y_train, y_test, results, FtDFres, kf, perfInd, epochs, batch, lstm_units, dense_units, dropout_percentage, loss_function, metric)
  res = results
  res.to_csv(exp + '.csv') 

Executing Experiment Average
Executing Experiment Average
Feature Extraction


  0%|          | 0/7067 [00:00<?, ?it/s]

Normalized features
Reducing features dimension
Dimensions reduced from (7067, 17) to (7067, 8)
Undersampling the majority class using ClusterCentroids
Majority class downsampled from (7027, 8) to (240, 8)
Oversampling the minority class using ADASYN
Minority class upsampled from (240, 8) to (406, 8)
Writing features to a csv file

Implementing SVM method...
Method: SVM
Accuracy: 0.8125
Sensitivity/Recall: 1.0
Specificity: 0.6
Precision: 0.7391304347826086
F1 Score: 0.85
Matthews Correlation Coefficient: 0.6659416347320276
Cohen’s Kappa: 0.6144578313253012
SVM finished in 0.01 sec

Implementing SVM k-fold...
Method: SVM Kfold
Accuracy: 0.8349593495934959
Sensitivity/Recall: 1.0
Specificity: 0.6623611111111111
Precision: 0.7497071935157041
F1 Score: 0.8554220053451089
Matthews Correlation Coefficient: 0.7036546130673583
Cohen’s Kappa: 0.6618962956423481
SVM k-fold finished in 0.1 sec

Implementing KNN...
Method: KNN
Accuracy: 0.9140625
Sensitivity/Recall: 0.9852941176470589
Specificity:

  0%|          | 0/7067 [00:00<?, ?it/s]

Normalized features
Reducing features dimension
Dimensions reduced from (7067, 34) to (7067, 15)
Undersampling the majority class using ClusterCentroids
Majority class downsampled from (7027, 15) to (240, 15)
Oversampling the minority class using ADASYN
Minority class upsampled from (240, 15) to (403, 15)
Writing features to a csv file

Implementing SVM method...
Method: SVM
Accuracy: 0.8740157480314961
Sensitivity/Recall: 1.0
Specificity: 0.7192982456140351
Precision: 0.813953488372093
F1 Score: 0.8974358974358974
Matthews Correlation Coefficient: 0.7651635878669806
Cohen’s Kappa: 0.7385486361296963
SVM finished in 0.01 sec

Implementing SVM k-fold...
Method: SVM Kfold
Accuracy: 0.841327160493827
Sensitivity/Recall: 1.0
Specificity: 0.6755099102064326
Precision: 0.7623748541544033
F1 Score: 0.86443317045839
Matthews Correlation Coefficient: 0.7165668120890916
Cohen’s Kappa: 0.6771618158684433
SVM k-fold finished in 0.06 sec

Implementing KNN...
Method: KNN
Accuracy: 0.937007874015748


  0%|          | 0/7067 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1684: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.obj[key] = infer_fill_value(value)


Normalized features
Reducing features dimension
Dimensions reduced from (7057, 289) to (7052, 90)
Undersampling the majority class using ClusterCentroids
Majority class downsampled from (7012, 90) to (240, 90)
Oversampling the minority class using ADASYN
Minority class upsampled from (240, 90) to (407, 90)
Writing features to a csv file

Implementing SVM method...
Method: SVM
Accuracy: 0.90625
Sensitivity/Recall: 1.0
Specificity: 0.8064516129032258
Precision: 0.8461538461538461
F1 Score: 0.9166666666666666
Matthews Correlation Coefficient: 0.8260642432614047
Cohen’s Kappa: 0.8112094395280236
SVM finished in 0.01 sec

Implementing SVM k-fold...
Method: SVM Kfold
Accuracy: 0.8673592291478471
Sensitivity/Recall: 1.0
Specificity: 0.7294871794871794
Precision: 0.7931526916083758
F1 Score: 0.8845522195764322
Matthews Correlation Coefficient: 0.760566280580526
Cohen’s Kappa: 0.7327950548934765
SVM k-fold finished in 0.06 sec

Implementing KNN...
Method: KNN
Accuracy: 0.9765625
Sensitivity/Rec

In [ ]:
df_full = pd.read_csv('Full.csv', delimiter = ',', header = 0)
df_avg = pd.read_csv('Average.csv', delimiter = ',', header = 0)
df_left_right= pd.read_csv('LeftRight.csv', delimiter = ',', header = 0)


In [ ]:
df_full

,Unnamed: 0,Accuracy,Sensitivity,Specificity,Precision,F1 Score,MCC,Kappa,Time
0,SVM,0.906250,1.000000,0.806452,0.846154,0.916667,0.826064,0.811209,0.01
1,SVM Kfold,0.867359,1.000000,0.729487,0.793153,0.884552,0.760566,0.732795,0.06
2,KNN,0.976562,0.984848,0.967742,0.970149,0.977444,0.953173,0.953056,0.02
3,KNN Kfold,0.950738,1.000000,0.901456,0.911375,0.952928,0.906363,0.900779,0.07
4,Logistic Regression,0.984375,1.000000,0.967742,0.970588,0.985075,0.969164,0.968689,0.03
5,Logistic Regression Kfold,0.940982,1.000000,0.877687,0.895886,0.944951,0.886645,0.880017,0.16
6,LSTM,0.984375,1.000000,0.967742,0.970588,0.985075,0.969164,0.968689,11.27
7,LSTM Kfold,0.997561,1.000000,0.995652,0.994595,0.997260,0.995123,0.995063,12.19


In [ ]:
df_avg

,Unnamed: 0,Accuracy,Sensitivity,Specificity,Precision,F1 Score,MCC,Kappa,Time
0,SVM,0.812500,1.000000,0.600000,0.739130,0.850000,0.665942,0.614458,0.01
1,SVM Kfold,0.834959,1.000000,0.662361,0.749707,0.855422,0.703655,0.661896,0.10
2,KNN,0.914062,0.985294,0.833333,0.870130,0.924138,0.834422,0.825915,0.01
3,KNN Kfold,0.906384,1.000000,0.807773,0.850953,0.917926,0.828840,0.811074,0.05
4,Logistic Regression,0.914062,1.000000,0.816667,0.860759,0.925170,0.838423,0.825570,0.01
5,Logistic Regression Kfold,0.904065,0.995238,0.813427,0.844637,0.912645,0.823637,0.808382,0.05
6,LSTM,0.929688,0.970588,0.883333,0.904110,0.936170,0.860806,0.858128,23.66
7,LSTM Kfold,0.933514,1.000000,0.863581,0.884584,0.938364,0.873918,0.865345,7.82


In [ ]:
df_left_right

,Unnamed: 0,Accuracy,Sensitivity,Specificity,Precision,F1 Score,MCC,Kappa,Time
0,SVM,0.874016,1.000000,0.719298,0.813953,0.897436,0.765164,0.738549,0.01
1,SVM Kfold,0.841327,1.000000,0.675510,0.762375,0.864433,0.716567,0.677162,0.06
2,KNN,0.937008,1.000000,0.859649,0.897436,0.945946,0.878339,0.871001,0.01
3,KNN Kfold,0.895772,0.995238,0.796410,0.833235,0.906041,0.809311,0.791519,0.05
4,Logistic Regression,0.929134,0.985714,0.859649,0.896104,0.938776,0.860596,0.855115,0.01
5,Logistic Regression Kfold,0.903302,0.995349,0.809351,0.840960,0.911374,0.819111,0.804720,0.06
6,LSTM,0.944882,0.985714,0.894737,0.920000,0.951724,0.890552,0.887682,11.73
7,LSTM Kfold,0.970309,1.000000,0.942270,0.942866,0.970289,0.942557,0.940399,8.49
